In [1]:
import rdkit
from rdkit import Chem
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import networkx as nx
import karateclub
from karateclub import DeepWalk
from karateclub import Graph2Vec
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder, AverageEmbedder


/home/jai/envs/ml4ns/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def generate_fingerprints(final_edges, fingerprint='RDKIT', graphembed='Deepwalk'):
    

    

    smiles = []
    for i in final_edges:
        smiles.append(i[0])
        smiles.append(i[1])
    smiles = np.array(list(set(smiles)))
    obj = []
    for i in tqdm(final_edges):
        if i[2] == '1':
            obj.append((np.where(smiles == i[0])[0][0],np.where(smiles == i[1])[0][0]) )

#     graph embeds using deepwalk

    G = nx.Graph()
    G.add_edges_from(obj)
    graph_embeds_data = []
    
    if graphembed == 'Deepwalk':

        model = DeepWalk()
        model.fit(G)
        embedding = model.get_embedding()

        for i in tqdm(final_edges):
            graph_embeds_data.append(np.array([ embedding[np.where(smiles == i[0])[0][0]], 
                                        embedding[np.where(smiles == i[1])[0][0]], 
                                        i[2] ]))

# graph embeds using node2vec
    edge_embeds = []
    if graphembed == 'node2vec':
        node2vec = Node2Vec(G, dimensions=128, walk_length=30, num_walks=200, workers=4)
        model = node2vec.fit(window=10, min_count=1, batch_words=4) 
        edges_embs = HadamardEmbedder(keyed_vectors=model.wv)
        edges_kv = edges_embs.as_keyed_vectors()

        for edge in final_edges:
            smi1 = np.where(smiles == edge[0])[0][0]
            smi2 = np.where(smiles == edge[1])[0][0]
            edge_embeds.append(edges_embs[(str(smi1),str(smi2))])

    graph_to_vec_embeds = []
    if graphembed == 'graph2vec':
        for edge in tqdm(final_edges):
            mol1 = Chem.MolFromSmiles(edge[0])
            mol2 = Chem.MolFromSmiles(edge[1])
            G1 = nx.Graph()
            G2 = nx.Graph()
            for atom in mol1.GetAtoms():
                G1.add_node(atom.GetIdx(), atomic_num=atom.GetAtomicNum(), is_aromatic=atom.GetIsAromatic(), atom_symbol=atom.GetSymbol())
        
            for bond in mol1.GetBonds():
                G1.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), bond_type=bond.GetBondType())
            
            for atom in mol2.GetAtoms():
                G2.add_node(atom.GetIdx(), atomic_num=atom.GetAtomicNum(), is_aromatic=atom.GetIsAromatic(), atom_symbol=atom.GetSymbol())
        
            for bond in mol2.GetBonds():
                G2.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), bond_type=bond.GetBondType())
        
            model = Graph2Vec()
            model.fit([G1])
            G1_graph2vec = model.get_embedding()
            G1_graph2vec = np.array(G1_graph2vec[0])
            
            model = Graph2Vec()
            model.fit([G2])
            G2_graph2vec = model.get_embedding()
            G2_graph2vec = np.array(G2_graph2vec[0])
            
            graph_to_vec_embeds.append(np.array([G1_graph2vec,G2_graph2vec,edge[2]]))
#     finger prints using normal rdkit
    if fingerprint == 'RDKIT':
        data = []
        for edge in tqdm(final_edges):
            mol1 = Chem.MolFromSmiles(edge[0])
            mol2 = Chem.MolFromSmiles(edge[1])
            fp1 = np.array(rdkit.Chem.RDKFingerprint(mol1, maxPath=7, fpSize=512), dtype=object)
            fp2 = np.array(rdkit.Chem.RDKFingerprint(mol2, maxPath=7, fpSize=512), dtype=object)
            data.append(np.array([fp1,fp2,edge[2]], dtype=object))
    
        finger_print_data = np.array(data)
        
        
    if fingerprint == 'MORGAN':
        data = []
        for edge in tqdm(final_edges):
            mol1 = Chem.MolFromSmiles(edge[0])
            mol2 = Chem.MolFromSmiles(edge[1])
            fp1 = np.asarray(rdkit.Chem.AllChem.GetMorganFingerprintAsBitVect(mol1, radius=3, nBits=64), dtype=object)
            fp2 = np.asarray(rdkit.Chem.AllChem.GetMorganFingerprintAsBitVect(mol2, radius=3, nBits=64), dtype=object)
            data.append(np.array([fp1,fp2,edge[2]], dtype=object))
        
        finger_print_data = np.array(data)
        
#     final data   
    data = {}
    val = None
    val2= None
    for i, edge in tqdm(enumerate(final_edges)):
        #if fingerprint == 'RDKIT':
        val = finger_print_data[i][:2]
        
        if graphembed == 'Deepwalk':
            val2 = graph_embeds_data[i][:2]
        
        if graphembed == 'node2vec':
            val2 = edge_embeds[i]
            
        if graphembed == 'graph2vec':
            val2 = graph_to_vec_embeds[i]
            
        sample = {'smiles':final_edges[i][:2],
                  'fingerprint': val,
                  'graphembedding':val2 ,
                  'link':final_edges[i][2]}
        
        data[i] = sample
    
    return data   

In [5]:
final_edges = np.load('../datasets/final_edges.dump', allow_pickle=True)

In [6]:
data = generate_fingerprints(final_edges, graphembed='node2vec')

  0%|          | 0/87153 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/418 [00:00<?, ?it/s]

Generating edge features: 100%|██████████| 87571/87571.0 [00:00<00:00, 203885.32it/s]


  0%|          | 0/87153 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [34]:
a = data[0]['graphembedding']

In [30]:
a

array([0.0610585 , 0.01124409], dtype=float32)

In [35]:
import torch
from torch.utils.data import Dataset, DataLoader

In [44]:
class LinkDataset(Dataset):
    def __init__(self, data,  graphembed='Deepwalk', transform=None):
        self.data = data
        self.graphembed = graphembed
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        fingerprints = np.concatenate((item['fingerprint'][0], item['fingerprint'][1]), axis = 0)
        if self.graphembed == 'Deepwalk':
            graphembeds = np.concatenate((item['graphembedding'][0], item['graphembedding'][1]), axis = 0)
        
        if self.graphembed == 'node2vec':
            graphembeds = item['graphembedding']
                
        label = int(item['link'])
        fp, ge= torch.tensor(fingerprints.astype(float)), torch.tensor(graphembeds.astype(float))
        l= torch.zeros(2)
        l[label] = 1
        return fp.unsqueeze(0), ge.unsqueeze(0), l

In [45]:
dataset = LinkDataset(data, graphembed='node2vec')
dataloader = DataLoader(dataset, num_workers = 4, batch_size=32)

In [46]:
for fp, ge, label in dataloader:
    print(fp.shape)
    print(ge.shape)
    print(label.shape)
    break

torch.Size([32, 1, 1024])
torch.Size([32, 1, 128])
torch.Size([32, 2])
